In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.decomposition import TruncatedSVD


import xgboost as xgb
%matplotlib inline

In [2]:
train_root = './data/train_preliminary/'
test_root = './data/test/'

In [3]:
ad = pd.read_csv(train_root+'ad.csv')
click_log = pd.read_csv(train_root +'click_log.csv')
user = pd.read_csv(train_root+'user.csv')
click_log

,time,user_id,creative_id,click_times
0,9,30920,567330,1
1,65,30920,3072255,1
2,56,30920,2361327,1
3,6,309204,325532,1
4,59,309204,2746730,1
...,...,...,...,...
30082766,6,30920,228129,1
30082767,57,30920,2361397,1
30082768,26,30920,629802,1
30082769,86,30920,2713031,1


In [4]:
merge = pd.merge(click_log, ad, on='creative_id')
merge = pd.merge(merge, user, on='user_id')
merge

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry,age,gender
0,9,30920,567330,1,504423,30673,3,32638,319,2,1
1,65,30920,3072255,1,2642300,1261,2,6783,6,2,1
2,56,30920,2361327,1,2035918,1261,2,6783,6,2,1
3,52,30920,2361397,1,2035987,1261,2,6783,6,2,1
4,53,30920,2361397,1,2035987,1261,2,6783,6,2,1
...,...,...,...,...,...,...,...,...,...,...,...
30082766,16,482613,855154,8,753602,38,2,9432,57,3,1
30082767,16,482613,855155,2,753603,38,2,9432,57,3,1
30082768,16,482613,826118,2,728905,1545,2,21975,183,3,1
30082769,31,482613,1467980,1,1278574,1545,2,26815,183,3,1


In [7]:
merge.product_category = merge.product_category.astype(str)
group = merge.groupby('user_id').product_category.apply(list).reset_index()
group
# tar =[30920,309204]
# features=[]
# for uid in tar:
    
#     feature = merge.loc[merge.user_id==uid,'creative_id'].values
#     features.append(feature)
# features

,user_id,product_category
0,1,"[18, 2, 2, 2, 5, 5, 2, 18, 5, 2, 5, 2, 5]"
1,2,"[2, 2, 2, 2, 2, 2, 18, 18, 18, 2, 2, 2, 2, 2, ..."
2,3,"[2, 17, 18, 18, 2, 2, 18, 2, 17, 8, 17, 2, 2, ..."
3,4,"[2, 2, 17, 2, 2, 17, 2, 18, 17, 2, 7, 18, 17, ..."
4,5,"[2, 18, 18, 18, 18, 18, 2, 18, 2, 2, 5, 2, 18,..."
...,...,...
899995,899996,"[2, 2, 2, 18, 18, 2, 2, 18, 5, 18, 18, 18, 18,..."
899996,899997,"[2, 3, 2, 2, 2, 18, 3, 2, 18, 2, 5, 2, 2, 2, 2..."
899997,899998,"[17, 12, 17, 18, 18, 18, 12, 18, 17, 3, 18, 18..."
899998,899999,"[2, 2, 2, 2, 8, 18, 18, 18, 18, 18, 13, 18, 5,..."


In [8]:
group['product_category'] = group['product_category'].apply(lambda x:' '.join(x))
group

,user_id,product_category
0,1,18 2 2 2 5 5 2 18 5 2 5 2 5
1,2,2 2 2 2 2 2 18 18 18 2 2 2 2 2 2 18 18 2 2 18 ...
2,3,2 17 18 18 2 2 18 2 17 8 17 2 2 2 2 2 4 18 2 1...
3,4,2 2 17 2 2 17 2 18 17 2 7 18 17 18 2 2 17 2 2 ...
4,5,2 18 18 18 18 18 2 18 2 2 5 2 18 18 2 18 2 5 5...
...,...,...
899995,899996,2 2 2 18 18 2 2 18 5 18 18 18 18 18
899996,899997,2 3 2 2 2 18 3 2 18 2 5 2 2 2 2 2 2 2
899997,899998,17 12 17 18 18 18 12 18 17 3 18 18 18 18
899998,899999,2 2 2 2 8 18 18 18 18 18 13 18 5 18 5 2 5 17 5...


In [5]:
train = pd.read_csv(train_root+'train_all.csv',index_col = 0)
train

,user_id,creative_id,advertiser_id,industry,ad_id,product_id,product_category,click_times,adv_pro,adv_ind,industry2,age,gender,uni_col
0,1,209778 877468 821396 1683713 122032 71691 1940...,9702 29455 7293 14668 11411 14681 17189 367 44...,6 106 326 326 \N 326 73 217 64 245 238 245 6,188507 773445 724607 1458878 109959 66210 1678...,136 \N \N \N 1334 \N \N \N \N 64 1454 64 1261,2 5 5 5 2 18 5 5 18 2 2 2 2,1 1 1 1 1 1 1 1 1 1 1 1 2,136_9702 \N_29455 \N_7293 \N_14668 1334_11411 ...,6_9702 106_29455 326_7293 326_14668 \N_11411 3...,6 106 326 326 326 326 73 217 64 245 238 245 6,4,1,9702_136_6_2 29455_\N_106_5 7293_\N_326_5 1466...
1,2,63441 155822 39714 609050 441462 1266180 13069...,22885 10686 18562 25932 22885 34505 768 26006 ...,318 238 6 6 318 47 317 47 6 47 242 47 6 6 47 4...,58788 139702 38066 541125 392680 1107111 14495...,87 80 129 129 87 \N 1400 \N 1261 \N 111 \N \N ...,2 2 2 2 2 18 2 18 2 18 2 18 18 2 18 18 2 2 2 2...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,87_22885 80_10686 129_18562 129_25932 87_22885...,318_22885 238_10686 6_18562 6_25932 318_22885 ...,318 238 6 6 318 47 317 47 6 47 242 47 6 6 47 4...,10,1,22885_87_318_2 10686_80_238_2 18562_129_6_2 25...
2,3,661347 808612 593522 710859 825434 726940 3920...,32974 9877 17018 18492 14186 9058 8371 2336 39...,\N \N 322 322 \N 6 54 6 322 322 205 302 322 32...,586668 713448 527601 629278 728308 643108 3502...,36256 40905 1674 1674 35985 \N \N 1031 1786 22...,17 17 2 2 17 18 18 2 2 2 18 12 2 2 18 18 8 4 2...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,36256_32974 40905_9877 1674_17018 1674_18492 3...,\N_32974 \N_9877 322_17018 322_18492 \N_14186 ...,319 319 322 322 322 6 54 6 322 322 205 302 322...,7,2,32974_36256_\N_17 9877_40905_\N_17 17018_1674_...
3,4,39588 589886 574787 1892854 1962706 2264105 12...,19451 7976 13084 12130 23664 811 10172 13299 1...,238 25 248 \N \N \N \N \N 88 319 6 238 319 238...,37966 524312 511235 1638619 1698206 1953681 10...,1862 \N 2625 38743 41265 37758 39904 39422 \N ...,2 18 2 17 17 4 17 17 18 2 2 2 2 2 2 2 2 2 5 2 ...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,1862_19451 \N_7976 2625_13084 38743_12130 4126...,238_19451 25_7976 248_13084 \N_12130 \N_23664 ...,238 25 248 248 248 248 248 248 88 319 6 238 31...,5,1,19451_1862_238_2 7976_\N_25_18 13084_2625_248_...
4,5,296145 350759 24333 43235 852327 1054434 12964...,11882 992 22885 9706 38760 2862 17745 31552 22...,297 \N 318 6 322 6 288 322 319 238 322 322 203...,265971 314795 24966 41148 751113 925792 113378...,\N \N 87 136 \N 136 \N \N \N 1064 \N 2620 \N \...,5 8 2 2 18 2 5 18 18 2 18 2 5 5 18 2 2 18 18 2...,1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,\N_11882 \N_992 87_22885 136_9706 \N_38760 136...,297_11882 \N_992 318_22885 6_9706 322_38760 6_...,297 297 318 6 322 6 288 322 319 238 322 322 20...,4,1,11882_\N_297_5 992_\N_\N_8 22885_87_318_2 9706...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,899996,92193 654526 939409 1477926 1458800 931537 181...,18103 13019 9934 9934 28323 2286 2003 35127 20...,54 40 54 54 40 54 40 6 202 6 248 248 248 54,84196 581202 827341 1286930 1270969 820416 157...,\N \N \N \N \N \N \N 129 \N 1567 2625 1794 262...,18 18 18 18 18 18 18 2 5 2 2 2 2 18,1 1 1 1 1 1 1 1 1 1 1 1 1 1,\N_18103 \N_13019 \N_9934 \N_9934 \N_28323 \N_...,54_18103 40_13019 54_9934 54_9934 40_28323 54_...,54 40 54 54 40 54 40 6 202 6 248 248 248 54,5,1,18103_\N_54_18 13019_\N_40_18 9934_\N_54_18 99...
899996,899997,69204 30210 7400 60330 2804147 3011222 24333 1...,24952 24975 768 28874 14681 21185 22885 14681 ...,60 5 317 60 6 289 318 297 321 161 319 318 6 32...,63976 30156 8817 55948 2410684 2589756 24966 1...,26858 1377 1400 26858 \N \N 87 \N 1250 91 70 8...,3 2 2 3 18 5 2 18 2 2 2 2 2 2 2 2 2 2,1 1 2 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1,26858_24952 1377_24975 1400_768 26858_28874 \N...,60_24952 5_24975 317_768 60_28874 6_14681 289_...,60 5 317 60 6 289 318 297 321 161 319 318 6 32...,3,2,24952_26858_60_3 24975_1377_5_2 768_1400_317_2...
899997,8999

In [61]:
# tfv = TfidfVectorizer(max_features=10000, analyzer='word',token_pattern=r"(?u)\b\w+\b")#, ngram_range=(1, 2)
tfv = CountVectorizer(max_features=10000, analyzer='word',token_pattern=r"(?u)\b\w+\b")
tfv.fit(train.product_category.values)
# xtrain_tfv =  tfv.transform(strs) 


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=10000, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w+\\b',
                tokenizer=None, vocabulary=None)

In [49]:

tfv.vocabulary_
# xtrain_tfv#.toarray()

{'6': 286,
 '106': 8,
 '326': 248,
 'n': 325,
 '73': 300,
 '217': 127,
 '64': 291,
 '245': 158,
 '238': 150,
 '318': 239,
 '47': 272,
 '317': 238,
 '242': 155,
 '5': 275,
 '36': 261,
 '319': 240,
 '322': 244,
 '289': 206,
 '21': 119,
 '207': 116,
 '250': 164,
 '328': 250,
 '54': 280,
 '205': 114,
 '302': 222,
 '247': 160,
 '25': 163,
 '248': 161,
 '88': 314,
 '295': 213,
 '297': 215,
 '288': 205,
 '203': 112,
 '220': 131,
 '296': 214,
 '28': 196,
 '109': 11,
 '8': 306,
 '183': 92,
 '9': 316,
 '13': 34,
 '293': 211,
 '60': 287,
 '221': 132,
 '1': 0,
 '241': 154,
 '86': 312,
 '230': 142,
 '246': 159,
 '134': 39,
 '192': 101,
 '292': 210,
 '202': 111,
 '329': 251,
 '158': 64,
 '231': 143,
 '291': 209,
 '26': 174,
 '29': 207,
 '149': 55,
 '74': 301,
 '24': 152,
 '253': 167,
 '259': 173,
 '147': 53,
 '232': 144,
 '3': 218,
 '294': 212,
 '82': 308,
 '104': 6,
 '159': 65,
 '58': 284,
 '89': 315,
 '116': 19,
 '108': 10,
 '215': 125,
 '234': 146,
 '99': 324,
 '271': 187,
 '40': 266,
 '161': 68,

In [10]:
alldata = pd.merge(group,user,on='user_id')
alldata

,user_id,product_category,age,gender
0,1,18 2 2 2 5 5 2 18 5 2 5 2 5,4,1
1,2,2 2 2 2 2 2 18 18 18 2 2 2 2 2 2 18 18 2 2 18 ...,10,1
2,3,2 17 18 18 2 2 18 2 17 8 17 2 2 2 2 2 4 18 2 1...,7,2
3,4,2 2 17 2 2 17 2 18 17 2 7 18 17 18 2 2 17 2 2 ...,5,1
4,5,2 18 18 18 18 18 2 18 2 2 5 2 18 18 2 18 2 5 5...,4,1
...,...,...,...,...
899995,899996,2 2 2 18 18 2 2 18 5 18 18 18 18 18,5,1
899996,899997,2 3 2 2 2 18 3 2 18 2 5 2 2 2 2 2 2 2,3,2
899997,899998,17 12 17 18 18 18 12 18 17 3 18 18 18 18,4,2
899998,899999,2 2 2 2 8 18 18 18 18 18 13 18 5 18 5 2 5 17 5...,3,1


In [62]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.product_category.values, train.gender.values, 
                                                  stratify=train.gender.values, 
                                                  random_state=7, 
                                                  test_size=0.01, shuffle=True)
print (xtrain.shape)
print (xvalid.shape)
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

(891000,)
(9000,)


In [63]:
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [64]:
predictions = clf.predict(xvalid_tfv)
accuracy_score(yvalid,predictions)

0.6744444444444444

In [65]:
####
xtrain, xvalid, ytrain, yvalid = train_test_split(train.product_category.values, train.age.values, 
                                                  stratify=train.age.values, 
                                                  random_state=7, 
                                                  test_size=0.01, shuffle=True)
print (xtrain.shape)
print (xvalid.shape)
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

(891000,)
(9000,)


In [66]:
clfage = LogisticRegression(C=1.0)
clfage.fit(xtrain_tfv, ytrain)
predictions = clfage.predict(xvalid_tfv)
accuracy_score(yvalid,predictions)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.22822222222222222

In [ ]:
uni  TFIDF 0.889/0.372    CF 0.8903/0.3806      c
cre  TFIDF 0.885/0.3413   CF 0.882/0.348        c   gender t
adv  TFIDF 0.888/0.3734   CF 0.880/0.371        t
pro  TFIDF 0.844/0.293    CF 0.813/0.262        t
ind  TFIDF 0.8167/0.2786  CF 0.810/0.2724       t
ad   TFIDF 0.8878/0.3415  CF 0.885/0.3495       c   gender t 
pry  TFIDF 0.6784/0.2283  CF 0.674/0.2282       t

In [112]:
Tclick_log = pd.read_csv(test_root+'click_log.csv')

In [113]:
Tclick_log.creative_id = Tclick_log.creative_id.astype(str)
group = Tclick_log.groupby('user_id').creative_id.apply(list).reset_index()
group['creative_id'] = group['creative_id'].apply(lambda x:' '.join(x))
group

,user_id,creative_id
0,3000001,351878 1508864 1711578 2259756 1797787 593698 ...
1,3000002,4131664 2222344 176984 1837242 1442051 47512 3...
2,3000003,1681016 3161357 2035400 3161357 4065209 360791...
3,3000004,9870 849706 1145367 108656 2944102 1328401 302...
4,3000005,3863936 3009874 71219 49682 183003 394403 1259...
...,...,...
999995,3999996,962662 648402 1837000 1103677 2423490 1369223 ...
999996,3999997,4249364 2227294 50445 390953 271727 882346 149...
999997,3999998,24188 1080706 54150 3647980 2188843 771243 352...
999998,3999999,545839 2560880 918536 3034248 1108217 3433843 ...


In [139]:
test_tfv =  tfv.transform(group['creative_id'].values)

In [135]:
predicted_gender = clf.predict(test_tfv)
predicted_gender

array([1, 2, 1, ..., 1, 1, 1])

In [136]:
predicted_age = clfage.predict(test_tfv)
predicted_age

array([3, 6, 4, ..., 2, 2, 2])

In [137]:
submission = pd.DataFrame(index =group.user_id )
# submission.index.name = 'user_id'
submission['predicted_age'] = predicted_age
submission['predicted_gender'] = predicted_gender
submission

,predicted_age,predicted_gender
user_id,,
3000001,3,1
3000002,6,2
3000003,4,1
3000004,1,1
3000005,5,1
...,...,...
3999996,4,1
3999997,2,1
3999998,2,1


In [138]:
submission.to_csv('submission.csv')

In [123]:
 import multiprocessing
multiprocessing.cpu_count()

48